In [31]:
from pathlib import Path

from tqdm import tqdm
import numpy as np
import pandas as pd

In [25]:
RAW_DATA_DIR = Path("data/raw/amazon")

In [3]:
meta = pd.read_pickle(RAW_DATA_DIR / "meta_Movies_and_TV.p")

In [5]:
meta.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Movies & TV, Movies]",,[],,Understanding Seizures and Epilepsy,[],[],,,[],"886,503 in Movies & TV (",[],Movies & TV,,,,0000695009,NaN
1,"[Movies & TV, Movies]",,[],,Spirit Led&mdash;Moving By Grace In The Holy S...,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"342,688 in Movies & TV (",[],Movies & TV,,,,0000791156,NaN
2,"[Movies & TV, Movies]",,[Disc 1: Flour Power (Scones; Shortcakes; Sout...,,My Fair Pastry (Good Eats Vol. 9),[],[https://images-na.ssl-images-amazon.com/image...,,Alton Brown,[],"370,026 in Movies & TV (",[],Movies & TV,,,,0000143529,NaN
3,"[Movies & TV, Movies]",,[Barefoot Contessa Volume 2: On these three di...,,"Barefoot Contessa (with Ina Garten), Entertain...","[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1...",[],,Ina Garten,[],"342,914 in Movies & TV (","[B002I5GNW4, 0804187045, B009UY3W8O, 060960219...",Movies & TV,,,$74.95,0000143588,NaN
4,"[Movies & TV, Movies]",,[Rise and Swine (Good Eats Vol. 7) includes bo...,,Rise and Swine (Good Eats Vol. 7),"[B000P1CKES, B000NR4CRM]",[https://images-na.ssl-images-amazon.com/image...,,Alton Brown,[],"351,684 in Movies & TV (",[B0015SVNXY],Movies & TV,,,,0000143502,NaN


In [6]:
meta.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

In [15]:
meta.dtypes

category        object
tech1           object
description     object
fit             object
title           object
also_buy        object
image           object
tech2           object
brand           object
feature         object
rank            object
also_view       object
main_cat        object
similar_item    object
date            object
price           object
asin            object
details         object
dtype: object

In [8]:
meta.shape

(203766, 18)

In [19]:
meta['category'] = meta.category.apply(lambda x: np.nan if x == '' else x)

In [20]:
meta.category.isna().sum()

0

In [45]:
def process_category(category):
    df_list = []
    for i, cat in enumerate(tqdm(category)):        
        tmp_df = pd.DataFrame(cat).transpose()
        tmp_df['index'] = i
        df_list.append(tmp_df)
    return pd.concat(df_list)    

In [46]:
categories = process_category(meta.category)

100%|██████████| 203766/203766 [02:30<00:00, 1351.76it/s]


In [47]:
categories.shape

(203541, 8)

In [48]:
categories.head()

,0,1,index,2,3,4,5,6
0,Movies & TV,Movies,0,NaN,NaN,NaN,NaN,NaN
0,Movies & TV,Movies,1,NaN,NaN,NaN,NaN,NaN
0,Movies & TV,Movies,2,NaN,NaN,NaN,NaN,NaN
0,Movies & TV,Movies,3,NaN,NaN,NaN,NaN,NaN
0,Movies & TV,Movies,4,NaN,NaN,NaN,NaN,NaN


In [49]:
meta.category.shape

(203766,)

In [51]:
categories['index'].nunique()

203541

In [52]:
categories['index'].max()

203765

In [55]:
np.setdiff1d(np.arange(meta.category.shape[0]), categories['index'].values)

array([  1782,   3119,   3214,   3245,   3359,   3767,   4059,   4189,
         4235,   4336,   4908,   4985,   6381,   6706,   7603,   7673,
         7710,   8008,   8427,   8587,   8696,   8738,   8878,   8898,
         9023,   9314,   9601,   9682,   9699,   9700,   9829,   9914,
         9918,  10491,  10498,  10512,  11615,  11755,  11763,  11802,
        12218,  13683,  13864,  14905,  15216,  15435,  15879,  15884,
        16023,  16274,  16379,  16729,  17103,  17128,  17303,  17636,
        17655,  20661,  21182,  21194,  21469,  21548,  21569,  22042,
        22530,  22532,  22585,  22586,  24578,  24919,  25305,  25997,
        28113,  28116,  29870,  30102,  30891,  32899,  32901,  33683,
        35078,  36097,  37938,  38061,  41104,  41378,  41380,  41383,
        41384,  41526,  42783,  42820,  43969,  44457,  44459,  44512,
        44513,  46505,  46846,  47232,  47924,  50040,  50043,  51797,
        52029,  52818,  54826,  54828,  55610,  57005,  58024,  59865,
      

In [57]:
meta.category[105448]

[]

In [58]:
meta[meta.price == ''].shape

(93021, 18)